## Single Modality Analysis of Mouse Brain Dataset

## Loading package

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import torch
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
import sys

# the location of R (used for the mclust clustering)
os.environ['R_HOME'] = 'E:/R-4.3.1'
os.environ['R_USER'] = 'E:/anaconda/lib/site-packages/rpy2'

save_path = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Visualization/Mouse_Thymus/' 

import sys
sys.path.insert(0, 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Model/')
from preprocess import preprocessing
from utils import mclust_R

## Loading and Preprocessing data

In [2]:
file_fold_1 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Mouse_Brain_ATAC/'
file_fold_2 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Mouse_Brain_H3K4me3/'
file_fold_3 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Mouse_Brain_H3K27ac/'
file_fold_4 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Mouse_Brain_H3K27me3/'

adata_omics_1_1 = sc.read_h5ad(file_fold_1 + 'adata_RNA.h5ad')
adata_omics_1_2 = sc.read_h5ad(file_fold_1 + 'adata_peaks_normalized.h5ad')

adata_omics_2_1 = sc.read_h5ad(file_fold_2 + 'adata_RNA.h5ad')
adata_omics_2_2 = sc.read_h5ad(file_fold_2 + 'adata_peaks_normalized.h5ad')

adata_omics_3_1 = sc.read_h5ad(file_fold_3 + 'adata_RNA.h5ad')
adata_omics_3_2 = sc.read_h5ad(file_fold_3 + 'adata_peaks_normalized.h5ad')

adata_omics_4_1 = sc.read_h5ad(file_fold_4 + 'adata_RNA.h5ad')
adata_omics_4_2 = sc.read_h5ad(file_fold_4 + 'adata_peaks_normalized.h5ad')

adata_omics_1_1.var_names_make_unique()
adata_omics_1_2.var_names_make_unique()
adata_omics_2_1.var_names_make_unique()
adata_omics_2_2.var_names_make_unique()
adata_omics_3_1.var_names_make_unique()
adata_omics_3_2.var_names_make_unique()
adata_omics_4_1.var_names_make_unique()
adata_omics_4_2.var_names_make_unique()

adata_omics_1_1, adata_omics_1_2 = preprocessing(adata_omics_1_1, adata_omics_1_2, 'Spatial-epigenome-transcriptome')
adata_omics_2_1, adata_omics_2_2 = preprocessing(adata_omics_2_1, adata_omics_2_2, 'Spatial-epigenome-transcriptome')
adata_omics_3_1, adata_omics_3_2 = preprocessing(adata_omics_3_1, adata_omics_3_2, 'Spatial-epigenome-transcriptome')
adata_omics_4_1, adata_omics_4_2 = preprocessing(adata_omics_4_1, adata_omics_4_2, 'Spatial-epigenome-transcriptome')

Spatial-epigenome-transcriptome data preprocessing have done!
Dimensions after preprocessed adata_modal_1: (9196, 3000)
Dimensions after preprocessing adata_modal_2: (9196, 121068)
Spatial-epigenome-transcriptome data preprocessing have done!
Dimensions after preprocessed adata_modal_1: (9513, 3000)
Dimensions after preprocessing adata_modal_2: (9513, 35270)
Spatial-epigenome-transcriptome data preprocessing have done!
Dimensions after preprocessed adata_modal_1: (9323, 3000)
Dimensions after preprocessing adata_modal_2: (9323, 104162)
Spatial-epigenome-transcriptome data preprocessing have done!
Dimensions after preprocessed adata_modal_1: (9732, 3000)
Dimensions after preprocessing adata_modal_2: (9732, 70470)


## Single Modality Clustering

In [3]:
## 分析的切片
adata_RNA_analysis = adata_omics_1_1
adata_ATAC_analysis = adata_omics_1_2

In [4]:
## 计算PCA后的RNA
sc.pp.pca(adata_RNA_analysis)
sc.pp.neighbors(adata_RNA_analysis, use_rep='X_pca')

## 计算PCA后的ADT
sc.pp.pca(adata_ATAC_analysis)
sc.pp.neighbors(adata_ATAC_analysis, use_rep='X_pca')

In [ ]:
# resolution combinations, ensuring appropriate number of clusters
# 1.2   0.9
# 0.83  1.4
# 1     1
# 0.85  1.35
sc.tl.leiden(adata_RNA_analysis, key_added="clusters_leiden", resolution=1.2)
sc.tl.leiden(adata_ATAC_analysis, key_added="clusters_leiden", resolution=0.9)
print(adata_RNA_analysis.obs['clusters_leiden'].unique())
print(adata_ATAC_analysis.obs['clusters_leiden'].unique())

['3', '2', '1', '4', '11', ..., '9', '7', '0', '10', '12']
Length: 13
Categories (13, object): ['0', '1', '2', '3', ..., '9', '10', '11', '12']
['2', '7', '0', '4', '3', ..., '1', '11', '6', '10', '12']
Length: 13
Categories (13, object): ['0', '1', '2', '3', ..., '9', '10', '11', '12']


In [5]:
results = sc.read_h5ad('D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Mouse_Brain_ATAC.h5ad')
results

AnnData object with n_obs × n_vars = 9196 × 0
    obs: 'SpaKnit', 'SpatialGlue', 'SpaGCN', 'STAGATE', 'Modality1', 'Modality2', 'MultiMAP', 'MultiVI'
    obsm: 'Modality1', 'Modality2', 'MultiMAP', 'MultiVI', 'STAGATE', 'SpaKnit', 'SpatialGlue', 'spatial'

In [ ]:
# results.obs['Modality1'] = adata_RNA_analysis.obs["clusters_leiden"].values
# results.obsm['Modality1'] = adata_RNA_analysis.obsm['X_pca']
# results.obs['Modality2'] = adata_ATAC_analysis.obs["clusters_leiden"].values
# results.obsm['Modality2'] = adata_ATAC_analysis.obsm['X_pca']
# results.write_h5ad('D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Mouse_Brain_ATAC.h5ad')

In [ ]:
## RNA Modality
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))
colors = [
    '#52b69a', '#e07a5f', '#d3edbe',  '#f4a259', '#e6db9e', '#d5a376', '#f9c74f', '#597b84',
    '#F19C79', '#ffbb78', '#98df8a', '#83c5be', '#a38161'
]
sc.pl.embedding(adata_ATAC_analysis, basis='spatial', color='clusters_leiden', ax=ax, s=70, show=False, palette=colors)
ax.set_title('RNA analysis')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')
# remove legend
ax.get_legend().remove()

# plt.savefig(save_path + 'ATAC1_RNA.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'ATAC1_RNA.eps', bbox_inches='tight', dpi=500)

In [ ]:
## RNA UMAP
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.pl.umap(adata_RNA_analysis, color='clusters_leiden', ax=ax,legend_loc='on data',legend_fontoutline=5, show=False)
ax.set_title('UMAP visulization')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')
# plt.savefig(save_path + 'ATAC_RNA_UMAP.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'ATAC_RNA_UMAP.eps', bbox_inches='tight', dpi=500)

In [ ]:
# # RNA PAGA
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.tl.paga(adata_RNA_analysis, groups='clusters_leiden')
sc.pl.paga(adata_RNA_analysis, edge_width_scale=1, node_size_scale=3, ax=ax, show=False, threshold=0.4, fontoutline=3)
ax.set_title('PAGA graph')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')

# plt.savefig(save_path + 'ATAC1_RNA_PAGA.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'ATAC1_RNA_PAGA.eps', bbox_inches='tight', dpi=500)

In [ ]:
## ATAC Modality
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))
colors = [
    '#52b69a', '#e07a5f', '#597b84',  '#f4a259', '#e6db9e', '#98df8a', '#83c5be', '#d3edbe',
    '#F19C79', '#a38161', '#d5a376', '#f9c74f', '#ffbb78'
]

sc.pl.embedding(adata_ATAC_analysis, basis='spatial', color='clusters_leiden', ax=ax, s=70, palette=colors, show=False)
ax.set_title('ATAC analysis')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')
# remove legend
ax.get_legend().remove()

# plt.savefig(save_path + 'ATAC1_ATAC.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'ATAC1_ATAC.eps', bbox_inches='tight', dpi=500)

In [ ]:
## RNA UMAP
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.pl.umap(adata_ATAC_analysis, color='clusters_leiden', ax=ax,legend_loc='on data',legend_fontoutline=5, show=False)
ax.set_title('UMAP visulization')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')

# plt.savefig(save_path + 'ATAC1_ATAC_UMAP.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'ATAC1_ATAC_UMAP.eps', bbox_inches='tight', dpi=500)

In [ ]:
# # RNA PAGA
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.tl.paga(adata_ATAC_analysis, groups='clusters_leiden')
sc.pl.paga(adata_ATAC_analysis, edge_width_scale=2, node_size_scale=5, ax=ax, show=False, threshold=0.2, fontoutline=3)
ax.set_title('PAGA graph')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')

# plt.savefig(save_path + 'ATAC1_ATAC_PAGA.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'ATAC1_ATAC_PAGA.eps', bbox_inches='tight', dpi=500)